In [1]:
import numpy as np
import ctypes

def coords_to_indices(coords, width):
    return coords[:, 1] * width + coords[:, 0]

def indices_to_coords(indices, width):
    y = indices // width
    x = indices % width
    return np.vstack((x, y)).T

a = np.array([(3, 6), (3, 7), (9, 9)])
b = np.array([(3, 6), (4, 6), (6, 6)])
w = 10

list1_indices = coords_to_indices(a, w).astype(np.int32)
list2_indices = coords_to_indices(b, w).astype(np.int32)

lib = ctypes.CDLL('./findIntersection.so')

# Define argument types and return type for the CUDA function
lib.launchFindIntersection.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.int32),  # list1
    np.ctypeslib.ndpointer(dtype=np.int32),  # list2
    ctypes.c_int,  # list1Size
    ctypes.c_int,  # list2Size
    np.ctypeslib.ndpointer(dtype=np.int32)   # result
]
lib.launchFindIntersection.restype = ctypes.c_int

result = np.zeros(len(list1_indices) + len(list2_indices), dtype=np.int32)

# Call the CUDA function and get the size of the union
union_size = lib.launchFindIntersection(
    list1_indices,
    list2_indices,
    len(list1_indices),
    len(list2_indices),
    result
)

# Slice the result array to get the actual union
union_result = result[:union_size]

print(union_result)

[63 73 99 64 66]
